In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# json파일들 -> csv파일 통합

In [ ]:
import json
import zipfile
import pandas as pd
myzip_r = zipfile.ZipFile('/content/drive/MyDrive/데이터/파/train/[라벨]파_0.정상.zip', 'r')
name_list = myzip_r.namelist()
df = pd.DataFrame()
for name in name_list:
    with myzip_r.open(name) as file:
        data = json.load(file)
        df2 = pd.DataFrame(data.values())
        df = pd.concat([df,df2])
# 이제 data_list에는 모든 파일의 데이터가 포함되어 있습니다.
df.loc[:,['disease', 'crop', 'area', 'grow', 'risk', 'points']] = df[['disease', 'crop', 'area', 'grow', 'risk', 'points']].shift(-1)
odd = []
for i in range(0,int(df.shape[0]/2)):
    odd.append(2*i)
data1 = df.iloc[odd,:]
data1.loc[:,'status'] = 0

<ipython-input-2-c2e2b0ccd2e0>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.loc[:,'status'] = 0


In [ ]:
import json
import zipfile
import pandas as pd
myzip_r = zipfile.ZipFile('/content/drive/MyDrive/데이터/파/train/[라벨]파_1.질병.zip', 'r')
name_list = myzip_r.namelist()
df = pd.DataFrame()
for name in name_list:
    with myzip_r.open(name) as file:
        data = json.load(file)
        df2 = pd.DataFrame(data.values())
        df = pd.concat([df,df2])
# 이제 data_list에는 모든 파일의 데이터가 포함되어 있습니다.
df.loc[:,['disease', 'crop', 'area', 'grow', 'risk', 'points']] = df[['disease', 'crop', 'area', 'grow', 'risk', 'points']].shift(-1)
odd = []
for i in range(0,int(df.shape[0]/2)):
    odd.append(2*i)
data2 = df.iloc[odd,:]
data2.loc[:,'status'] = 1

<ipython-input-3-67c9637c439e>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2.loc[:,'status'] = 1


In [ ]:
import json
import zipfile
import pandas as pd
myzip_r = zipfile.ZipFile('/content/drive/MyDrive/데이터/파/train/[라벨]파_9.증강.zip', 'r')
name_list = myzip_r.namelist()
df = pd.DataFrame()
for name in name_list:
    with myzip_r.open(name) as file:
        data = json.load(file)
        df2 = pd.DataFrame(data.values())
        df = pd.concat([df,df2])
df.loc[:,['disease', 'crop', 'area', 'grow', 'risk', 'points']] = df[['disease', 'crop', 'area', 'grow', 'risk', 'points']].shift(-1)
df.loc[:,['brightness', 'contrast', 'saturation', 'hue', 'transpose', 'hflip', 'vflip']] = df[['brightness', 'contrast', 'saturation', 'hue', 'transpose', 'hflip','vflip']].shift(-2)
odd = []
for i in range(0,int(df.shape[0]/3)):
    odd.append(3*i)
print(len(odd))
data3 = df.iloc[odd,:]
data3.loc[:,'status'] = 9

31755


<ipython-input-4-29d52ed443e8>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3.loc[:,'status'] = 9


In [ ]:
print(data1.shape,data2.shape,data3.shape)
data = pd.concat([data1,data2,data3])
data.info()

(11372, 15) (2117, 15) (31755, 23)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 45244 entries, 0 to 0
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   image       45244 non-null  object 
 1   date        45244 non-null  object 
 2   worker      45244 non-null  object 
 3   height      45244 non-null  float64
 4   width       45244 non-null  float64
 5   task        45244 non-null  float64
 6   type        45244 non-null  float64
 7   region      33872 non-null  float64
 8   disease     45244 non-null  float64
 9   crop        45244 non-null  float64
 10  area        45244 non-null  float64
 11  grow        45244 non-null  float64
 12  risk        45244 non-null  float64
 13  points      45244 non-null  object 
 14  status      45244 non-null  int64  
 15  original    31755 non-null  object 
 16  brightness  31755 non-null  float64
 17  contrast    31755 non-null  float64
 18  saturation  31755 non-null  float64

In [ ]:
data.drop('region',axis=1,inplace=True)

In [ ]:
data.to_csv('[라벨]파.csv',index=False)

# 넘파이 파일 불러와서 그레이스케일 변환 후 데이터분할

In [ ]:
!pip install Pillow

In [ ]:
import os,sys
from PIL import Image
def resize_and_crop(img_array,size):
    img = Image.fromarray(image_array)
    img_ratio = img.size[0] / float(img.size[1])
    ratio = size[0] / float(size[1])

    if ratio > img_ratio:
        img = img.resize((size[0], int(round(size[0] * img.size[1] / img.size[0]))),
            Image.LANCZOS)
        box = (0, int(round((img.size[1] - size[1]) / 2)), img.size[0],
               int(round((img.size[1] + size[1]) / 2)))
        img = img.crop(box)

    elif ratio < img_ratio:
        img = img.resize((int(round(size[1] * img.size[0] / img.size[1])), size[1]),
            Image.LANCZOS)
        box = (int(round((img.size[0] - size[0]) / 2)), 0,
                int(round((img.size[0] + size[0]) / 2)), img.size[1])
        img = img.crop(box)

    else :
        img = img.resize((size[0], size[1]), Image.LANCZOS)

    return img

In [ ]:
import pandas as pd
import zipfile
import numpy as np
from PIL import Image

# ZIP 파일 경로
zip_file_path = '/content/drive/MyDrive/데이터/파/train/[원천]파_0.정상.zip'
i = 1
# ZIP 파일 열기
with zipfile.ZipFile(zip_file_path, 'r') as z:
    # ZIP 내의 파일 목록을 가져옴
    name_list = z.namelist()
    image_arrays = []
    for name in name_list:
        with z.open(name) as file:
            image = Image.open(file)
            # 이미지를 numpy 배열로 변환
            image_array = np.array(image)
            output_size = (224, 224)
            # 이미지 축소
            resized_img = resize_and_crop(image_array,output_size)
            # 그레이스케일
            img = np.array(resized_img).reshape(224, 224, 3)
            image_arrays.append(img)
        print(f'{len(name_list)}중에 {i}번 성공')
        i += 1
# 리스트의 numpy 배열들을 4차원 numpy 배열로 변환
image_arr = np.stack(image_arrays)
file_path = '/content/drive/MyDrive/데이터/파/train/파_정상_224_color.npy'
np.save(file_path, image_arr)
# 결과 확인
print(image_arr.shape)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
11372중에 6374번 성공
11372중에 6375번 성공
11372중에 6376번 성공
11372중에 6377번 성공
11372중에 6378번 성공
11372중에 6379번 성공
11372중에 6380번 성공
11372중에 6381번 성공
11372중에 6382번 성공
11372중에 6383번 성공
11372중에 6384번 성공
11372중에 6385번 성공
11372중에 6386번 성공
11372중에 6387번 성공
11372중에 6388번 성공
11372중에 6389번 성공
11372중에 6390번 성공
11372중에 6391번 성공
11372중에 6392번 성공
11372중에 6393번 성공
11372중에 6394번 성공
11372중에 6395번 성공
11372중에 6396번 성공
11372중에 6397번 성공
11372중에 6398번 성공
11372중에 6399번 성공
11372중에 6400번 성공
11372중에 6401번 성공
11372중에 6402번 성공
11372중에 6403번 성공
11372중에 6404번 성공
11372중에 6405번 성공
11372중에 6406번 성공
11372중에 6407번 성공
11372중에 6408번 성공
11372중에 6409번 성공
11372중에 6410번 성공
11372중에 6411번 성공
11372중에 6412번 성공
11372중에 6413번 성공
11372중에 6414번 성공
11372중에 6415번 성공
11372중에 6416번 성공
11372중에 6417번 성공
11372중에 6418번 성공
11372중에 6419번 성공
11372중에 6420번 성공
11372중에 6421번 성공
11372중에 6422번 성공
11372중에 6423번 성공
11372중에 6424번 성공
11372중에 6425번 성공
11372중에 6426번 성공
11372중에 6427번 성공
11372중에 6428번 성공
11372중에 6429번 성공
11372중에 6430

In [ ]:
import pandas as pd
import zipfile
import numpy as np
from PIL import Image

# ZIP 파일 경로
zip_file_path = '/content/drive/MyDrive/데이터/파/train/[원천]파_1.질병.zip'
i = 1
# ZIP 파일 열기
with zipfile.ZipFile(zip_file_path, 'r') as z:
    # ZIP 내의 파일 목록을 가져옴
    name_list = z.namelist()
    image_arrays = []
    for name in name_list:
        with z.open(name) as file:
            image = Image.open(file)
            # 이미지를 numpy 배열로 변환
            image_array = np.array(image)
            output_size = (224, 224)
            # 이미지 축소
            resized_img = resize_and_crop(image_array,output_size)
            # 그레이스케일
            img = np.array(resized_img).reshape(224,224,3)
            image_arrays.append(img)
        print(f'{len(name_list)}중에 {i}번 성공')
        i += 1
# 리스트의 numpy 배열들을 4차원 numpy 배열로 변환
image_arr = np.stack(image_arrays)
file_path = '/content/drive/MyDrive/데이터/파/train/파_질병_224_color.npy'
np.save(file_path, image_arr)
# 결과 확인
print(image_arr.shape)


2117중에 1번 성공
2117중에 2번 성공
2117중에 3번 성공
2117중에 4번 성공
2117중에 5번 성공
2117중에 6번 성공
2117중에 7번 성공
2117중에 8번 성공
2117중에 9번 성공
2117중에 10번 성공
2117중에 11번 성공
2117중에 12번 성공
2117중에 13번 성공
2117중에 14번 성공
2117중에 15번 성공
2117중에 16번 성공
2117중에 17번 성공
2117중에 18번 성공
2117중에 19번 성공
2117중에 20번 성공
2117중에 21번 성공
2117중에 22번 성공
2117중에 23번 성공
2117중에 24번 성공
2117중에 25번 성공
2117중에 26번 성공
2117중에 27번 성공
2117중에 28번 성공
2117중에 29번 성공
2117중에 30번 성공
2117중에 31번 성공
2117중에 32번 성공
2117중에 33번 성공
2117중에 34번 성공
2117중에 35번 성공
2117중에 36번 성공
2117중에 37번 성공
2117중에 38번 성공
2117중에 39번 성공
2117중에 40번 성공
2117중에 41번 성공
2117중에 42번 성공
2117중에 43번 성공
2117중에 44번 성공
2117중에 45번 성공
2117중에 46번 성공
2117중에 47번 성공
2117중에 48번 성공
2117중에 49번 성공
2117중에 50번 성공
2117중에 51번 성공
2117중에 52번 성공
2117중에 53번 성공
2117중에 54번 성공
2117중에 55번 성공
2117중에 56번 성공
2117중에 57번 성공
2117중에 58번 성공
2117중에 59번 성공
2117중에 60번 성공
2117중에 61번 성공
2117중에 62번 성공
2117중에 63번 성공
2117중에 64번 성공
2117중에 65번 성공
2117중에 66번 성공
2117중에 67번 성공
2117중에 68번 성공
2117중에 69번 성공
2117중에 70번 성공
2117중에 71번 성공
2117중에 72번 성공
2

In [ ]:
import pandas as pd
import zipfile
import numpy as np
from PIL import Image

# ZIP 파일 경로
zip_file_path = '/content/drive/MyDrive/데이터/파/train/[원천]파_9.증강.zip'
i = 1
# ZIP 파일 열기
with zipfile.ZipFile(zip_file_path, 'r') as z:
    # ZIP 내의 파일 목록을 가져옴
    name_list = z.namelist()
    image_arrays = []
    for name in name_list:
        with z.open(name) as file:
            image = Image.open(file)
            # 이미지를 numpy 배열로 변환
            image_array = np.array(image)
            output_size = (224,224)
            # 이미지 축소
            resized_img = resize_and_crop(image_array,output_size)
            # 그레이스케일
            img = np.array(resized_img).reshape(224,224,3)
            image_arrays.append(img)
        print(f'{len(name_list)}중에 {i}번 성공')
        i += 1
# 리스트의 numpy 배열들을 4차원 numpy 배열로 변환
image_arr = np.stack(image_arrays)
file_path = '/content/drive/MyDrive/데이터/파/train/파_증강_224_color.npy'
np.save(file_path, image_arr)
# 결과 확인
print(image_arr.shape)


Streaming output truncated to the last 5000 lines.
31755중에 26757번 성공
31755중에 26758번 성공
31755중에 26759번 성공
31755중에 26760번 성공
31755중에 26761번 성공
31755중에 26762번 성공
31755중에 26763번 성공
31755중에 26764번 성공
31755중에 26765번 성공
31755중에 26766번 성공
31755중에 26767번 성공
31755중에 26768번 성공
31755중에 26769번 성공
31755중에 26770번 성공
31755중에 26771번 성공
31755중에 26772번 성공
31755중에 26773번 성공
31755중에 26774번 성공
31755중에 26775번 성공
31755중에 26776번 성공
31755중에 26777번 성공
31755중에 26778번 성공
31755중에 26779번 성공
31755중에 26780번 성공
31755중에 26781번 성공
31755중에 26782번 성공
31755중에 26783번 성공
31755중에 26784번 성공
31755중에 26785번 성공
31755중에 26786번 성공
31755중에 26787번 성공
31755중에 26788번 성공
31755중에 26789번 성공
31755중에 26790번 성공
31755중에 26791번 성공
31755중에 26792번 성공
31755중에 26793번 성공
31755중에 26794번 성공
31755중에 26795번 성공
31755중에 26796번 성공
31755중에 26797번 성공
31755중에 26798번 성공
31755중에 26799번 성공
31755중에 26800번 성공
31755중에 26801번 성공
31755중에 26802번 성공
31755중에 26803번 성공
31755중에 26804번 성공
31755중에 26805번 성공
31755중에 26806번 성공
31755중에 26807번 성공
31755중에 26808번 성공
31755중에 26809

In [ ]:
import numpy as np
import pandas as pd
normal = np.load('/content/drive/MyDrive/데이터/파/train/파_정상_224.npy')
disease = np.load('/content/drive/My Drive/데이터/파/train/파_질병_224.npy')
expend = np.load('/content/drive/My Drive/데이터/파/train/파_증강_224.npy')
diseases = np.concatenate((disease, expend), axis=0)
X_train2 = np.concatenate((normal, diseases), axis=0)
labels = pd.read_csv('/content/drive/MyDrive/데이터/파/train/[라벨]파.csv')
if X_train2.shape[0]==labels.shape[0] :
    y_train = labels.loc[:,'disease']
else :
    print('길이가 같지 않습니다.')

In [ ]:
np.unique(y_train,return_counts=True)

In [ ]:
y_train[y_train==16]=1
y_train[y_train==17]=2
y_train[y_train==18]=3
np.unique(y_train,return_counts=True)

In [ ]:
num = 4144
a = np.where(y_train == 0)[0][:num]
b = np.where(y_train == 1)[0][:num]
c = np.where(y_train == 2)[0][:num]
d = np.where(y_train == 3)[0][:num]
select = []
select.extend(a)
select.extend(b)
select.extend(c)
select.extend(d)
X_train = X_train2[select]
y_train = y_train[select]


In [ ]:
if y_train.shape[0] == num*4 :
    print(y_train.shape)
else :
    print('길이가 다릅니다.')

In [ ]:
X_train.shape

In [ ]:
np.save('/content/drive/MyDrive/데이터/파/X_train_224_equal.npy',X_train)
np.save('/content/drive/MyDrive/데이터/파/y_train_224_equal.npy',y_train)